In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import unicodedata
import re



In [2]:
filename = "feedback-history-export-1766510816456.json"


In [3]:
q_df = pd.read_json("cherrypick.jsonl", lines=True)

In [4]:
q_df = q_df.rename(columns={"answer": "reference_answer"})
q_df["docID"] = q_df["docID"].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x
)
q_df["queryID"] = range(len(q_df))



In [5]:
q_df

,query,docID,reference_answer,type,queryID
0,"Jak se jmenuje neuronová síť, která vznikla ja...",0,Grossbergova neuronová síť,direct,0
1,Jaký je hlavní rozdíl mezi úzkou a obecnou umě...,0,"Úzká AI řeší jedinou úzce vymezenou úlohu, zat...",direct,1
2,"Kdo byl autorem teorie umělé inteligence, jak ...",0,Článek neuvádí žádného autora teorie umělé int...,trick,2
3,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3
4,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4
5,Proč je v NLP nutná samostatná reprezentace pr...,4,Aby byla zajištěna přesnost a jednoznačnost re...,direct,5
6,Pro kterou společnost pracuje Greg Corrado pod...,5,Google,direct,6
7,Jaký matematický nástroj Word2Vec používá k ur...,5,úroveň sémantické podobnosti mezi slovy (pomoc...,paraphrase,7
8,Byl Word2Vec vyvinut výzkumníky z Microsoftu?,5,Model byl publikován výzkumníky Tomášem Mikolo...,trick,8
9,Jaký fonologický rys odlišuje foném /m/ od fon...,8,rys nazálnosti,direct,9


In [6]:
with open(filename, "r", encoding="utf-8") as f:
    logs = json.load(f)


In [7]:
events = logs

In [8]:
rows = []

for e in events:
    rating_value = e["data"].get("rating")
    rated_model = e["data"].get("model_id")  # ✅ KEY LINE

    rating_comment = e["data"].get("comment", "")
    rating_reason = e["data"].get("reason", "")
    sibling_models = e["data"].get("sibling_model_ids", [])

    chat = e["snapshot"]["chat"]
    chat_id = chat["id"]
    user_id = chat["user_id"]
    chat_title = chat["title"]

    messages = chat["chat"]["history"]["messages"]

    # find the USER prompt (parentId == None)
    user_msgs = [
        m for m in messages.values()
        if m["role"] == "user" and m["parentId"] is None
    ]

    for user_msg in user_msgs:
        prompt = user_msg["content"]
        children = user_msg.get("childrenIds", [])

        for child_id in children:
            if child_id not in messages:
                continue

            child = messages[child_id]
            if child["role"] != "assistant":
                continue

            answer_model = child.get("model")

            is_rated = (answer_model == rated_model)

            rows.append({
                "chat_id": chat_id,
                "chat_title": chat_title,
                "user_id": user_id,

                "prompt": prompt,
                "answer": child["content"],

                "model": answer_model,
                "model_idx": child.get("modelIdx"),

                "siblings": sibling_models,

                # ✅ CORRECT SEMANTICS
                "rating": rating_value if is_rated else 0,
                "rating_comment": rating_comment if is_rated else "",
                "rating_reason": rating_reason if is_rated else "",

                "is_rated_model": is_rated,
            })


In [9]:
df = pd.DataFrame(rows)
df = df[df["is_rated_model"] == True]
df.head(20)


,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,rating_reason,is_rated_model
1,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,other,True
4,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,,True
9,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza hraje důležitou...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,,True
12,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza v metodě automa...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,,True
19,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",-1,c - špatný příklad různých tvarů stejného slov...,,True
22,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,,True
25,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,Lemmatizace zlepšuje latentní sémantickou anal...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",-1,c - nečeské slova či části slov v textu,,True
28,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,Lemmatizace zlepšuje latentní sémantickou anal...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",-1,"c - špatný příklad tvarů jednoho slova, špatně...",,True
35,01d2a877-b5db-4f63-8c72-e6d1a9ae448a,Lemmatization of „barvě“,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaký výstup lemmatizátor vrátí pro tvar „barvě...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",1,,,True
38,01d2a877-b5db-4f63-8c72-e6d1a9ae448a,Lemmatization of „barvě“,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaký výstup lemmatizátor vrátí pro tvar „barvě...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",-1,f - podstatné jméno není mluvnická kategorie,,True


In [10]:
df["rating_zero_based"] = (df["rating"] + 1) // 2

In [11]:
df

,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,rating_reason,is_rated_model,rating_zero_based
1,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,other,True,1
4,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,,True,1
9,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza hraje důležitou...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,,True,1
12,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza v metodě automa...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,,True,1
19,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",-1,c - špatný příklad různých tvarů stejného slov...,,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,a937c44c-54e7-48b2-a886-a0e8f4707b76,🔍 Unique NLP Representations Needed,b07241e5-a921-4445-bdfc-a876a7ceb08e,Proč je v NLP nutná samostatná reprezentace pr...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,None,None,True,1
562,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,,True,1
567,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",1,"Něco navíc, rychlé",,True,1
571,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",1,,,True,1


In [12]:
df["prompt"].unique()

array(['Jaké jsou tři historické fáze zpracování přirozeného jazyka uvedené v článku?',
       'Jaké jsou úkoly morfologicko-lexikální analýzy v metodě automatického indexování zvané mozaika?',
       'Jak lemmatizace zlepšuje latentní sémantickou analýzu?',
       'Jaký výstup lemmatizátor vrátí pro tvar „barvě“ podle textu?',
       'Jaké doplňkové informace může lemmatizátor poskytovat?',
       'Proč je v NLP nutná samostatná reprezentace pro každý významový prvek přirozeného jazyka?',
       'Jaké jsou hlavní typy jazykových korpusů a jejich charakteristiky?',
       'Kdo založil Ústav Českého národního korpusu?',
       'Jaký značkovací jazyk se používá pro formátování textů v jazykových korpusech?',
       'Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?',
       'Jaký matematický nástroj Word2Vec používá k určení, jak moc jsou si slova podobná?',
       'Byl Word2Vec vyvinut výzkumníky z Microsoftu?',
       'Kdo byl 

In [13]:
def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return s

    # Unicode normalization
    s = unicodedata.normalize("NFKC", s)

    # Normalize hyphens/dashes
    s = re.sub(r"[‐-‒–—−]", "-", s)

    # Normalize quotes
    s = s.replace("„", '"').replace("“", '"').replace("”", '"')
    s = s.replace("’", "'").replace("‘", "'")

    # Normalize whitespace
    s = re.sub(r"\s+", " ", s).strip()

    return s


In [14]:
df["prompt_norm"] = df["prompt"].apply(normalize_text)
q_df["query_norm"] = q_df["query"].apply(normalize_text)


In [15]:
missing = set(df["prompt_norm"]) - set(q_df["query_norm"])
assert not missing, f"❌ Still missing: {missing}"


In [16]:
df = df.merge(
    q_df,
    left_on="prompt_norm",
    right_on="query_norm",
    how="left",
    validate="many_to_one"
)


In [17]:
df

,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,rating_reason,is_rated_model,rating_zero_based,prompt_norm,query,docID,reference_answer,type,queryID,query_norm
0,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,other,True,1,Jaké jsou tři historické fáze zpracování přiro...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...
1,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,,True,1,Jaké jsou tři historické fáze zpracování přiro...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...
2,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza hraje důležitou...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,,True,1,Jaké jsou úkoly morfologicko-lexikální analýzy...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
3,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza v metodě automa...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,,True,1,Jaké jsou úkoly morfologicko-lexikální analýzy...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
4,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",-1,c - špatný příklad různých tvarů stejného slov...,,True,0,Jak lemmatizace zlepšuje latentní sémantickou ...,Jak lemmatizace zlepšuje latentní sémantickou ...,17,snižuje počet slov na minimum převodem všech s...,direct,12,Jak lemmatizace zlepšuje latentní sémantickou ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,a937c44c-54e7-48b2-a886-a0e8f4707b76,🔍 Unique NLP Representations Needed,b07241e5-a921-4445-bdfc-a876a7ceb08e,Proč je v NLP nutná samostatná reprezentace pr...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,None,None,True,1,Proč je v NLP nutná samostatná reprezentace pr...,Proč je v NLP nutná samostatná reprezentace pr...,4,Aby byla zajištěna přesnost a jednoznačnost re...,direct,5,Proč je v NLP nutná samostatná reprezentace pr...
140,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,,True,1,Jaké jsou úkoly morfologicko-lexikální analýzy...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
141,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" durat

In [18]:
# check all users have same number of answers 4*6
df.user_id.value_counts()

user_id
b07241e5-a921-4445-bdfc-a876a7ceb08e    24
20bdbdd1-722a-48d4-aebc-d664dd6e8a95    24
2d07748f-d611-477a-8209-7f5a2d077003    24
c44521d7-31f7-48df-8c90-79e591240579    24
ecbb6d4e-b5b4-4cea-b1b7-2a4663bcc834    24
cf259990-90b0-45a7-93c4-31aec3610b29    24
Name: count, dtype: int64

In [19]:
df[df['user_id'] == '20bdbdd1-722a-48d4-aebc-d664dd6e8a95']['prompt'].value_counts()

prompt
Proč je v NLP nutná samostatná reprezentace pro každý významový prvek přirozeného jazyka?          4
Jaké jsou úkoly morfologicko-lexikální analýzy v metodě automatického indexování zvané mozaika?    4
Jaké jsou tři historické fáze zpracování přirozeného jazyka uvedené v článku?                      4
Jaké jsou hlavní typy jazykových korpusů a jejich charakteristiky?                                 4
Kdo založil Ústav Českého národního korpusu?                                                       4
Jaký značkovací jazyk se používá pro formátování textů v jazykových korpusech?                     4
Name: count, dtype: int64

In [20]:
import re

def parse_error_types(comment):
    """
    Returns:
      error_factual (0/1),
      error_czech (0/1),
      factual_comment (str),
      czech_comment (str)
    """
    if not isinstance(comment, str) or not comment.strip():
        return 0, 0, "", ""

    factual_lines = []
    czech_lines = []

    lines = comment.splitlines()

    for line in lines:
        l = line.strip()

        # match: f / - f / f: / -f / f -
        if re.match(r"^[-\s]*f\b[:\-]?\s*", l, re.IGNORECASE):
            factual_lines.append(l)

        # match: c / - c / c: / -c / c -
        if re.match(r"^[-\s]*c\b[:\-]?\s*", l, re.IGNORECASE):
            czech_lines.append(l)

    return (
        int(len(factual_lines) > 0),
        int(len(czech_lines) > 0),
        "\n".join(factual_lines),
        "\n".join(czech_lines),
    )


In [21]:
def parse_row(row):
    if row["rating"] != -1:
        return 0, 0, "", ""
    return parse_error_types(row["rating_comment"])


In [22]:
parsed = df.apply(parse_row, axis=1)

df[
    ["error_factual", "error_czech", "factual_comment", "czech_comment"]
] = pd.DataFrame(parsed.tolist(), index=df.index)


In [23]:
# positive ratings → clean
assert (df[df["rating"] == 1]["error_factual"] == 0).all()
assert (df[df["rating"] == 1]["error_czech"] == 0).all()

# negative ratings → something meaningful
df[df["rating"] == -1][
    ["rating_comment", "error_factual", "error_czech"]
]


,rating_comment,error_factual,error_czech
4,c - špatný příklad různých tvarů stejného slov...,0,1
6,c - nečeské slova či části slov v textu,0,1
7,"c - špatný příklad tvarů jednoho slova, špatně...",0,1
9,f - podstatné jméno není mluvnická kategorie,1,0
10,f - podstatné jméno není mluvnická kategorie,1,0
13,f - špatné kaegorie u příkladu sloves,1,0
14,f- slovní typ místo slovní druh,1,0
15,c - mathematický místo matemaický,0,1
16,c - čínský znak uprostřed,0,1
17,f - 4 není elevantní,1,0


In [24]:
orange = 'b07241e5-a921-4445-bdfc-a876a7ceb08e'
green = '2d07748f-d611-477a-8209-7f5a2d077003'
pink = ''
blue = ''
purple = ''
yellow = ''

In [25]:
model_ratings = (
    df
    .groupby("model")
    .agg(
        rating_mean=("rating_zero_based", "mean"),
        rating_count=("rating_zero_based", "sum"),
        error_factual_mean=("error_factual", "mean"),
        error_factual_sum=("error_factual", "sum"),
        error_czech_mean=("error_czech", "mean"),
        error_czech_sum=("error_czech", "sum"),


    )
    .sort_values("rating_mean", ascending=False)
)
model_ratings

,rating_mean,rating_count,error_factual_mean,error_factual_sum,error_czech_mean,error_czech_sum
model,,,,,,
llama3.3:latest,0.722222,26,0.111111,4,0.222222,8
gpt-oss-120b,0.694444,25,0.083333,3,0.222222,8
qwen3:32b,0.694444,25,0.166667,6,0.194444,7
phi4:14b-q8_0,0.277778,10,0.222222,8,0.611111,22


In [26]:
model_ratings.to_latex(
    "model_ratings.tex",
    float_format="%.3f",
    caption="Model comparison based on human evaluator ratings.",
    label="tab:generator_ratings",
    index=True,
    bold_rows=True,
    column_format="lcccccc",
    escape=False,
    longtable=False,
    multicolumn=True,
    multicolumn_format="c",
)


In [27]:
winners = df[df["rating"] == 1]


In [28]:
winners

,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,...,query,docID,reference_answer,type,queryID,query_norm,error_factual,error_czech,factual_comment,czech_comment
0,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...,0,0,,
1,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...,0,0,,
2,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza hraje důležitou...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
3,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza v metodě automa...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
5,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,...,Jak lemmatizace zlepšuje latentní sémantickou ...,17,snižuje počet slov na minimum převodem všech s...,direct,12,Jak lemmatizace zlepšuje latentní sémantickou ...,0,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,a937c44c-54e7-48b2-a886-a0e8f4707b76,🔍 Unique NLP Representations Needed,b07241e5-a921-4445-bdfc-a876a7ceb08e,Proč je v NLP nutná samostatná reprezentace pr...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,None,...,Proč je v NLP nutná samostatná reprezentace pr...,4,Aby byla zajištěna přesnost a jednoznačnost re...,direct,5,Proč je v NLP nutná samostatná reprezentace pr...,0,0,,
140,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
141,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",1,"Něco navíc, rychlé",...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
142,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fá

In [29]:
losers = df[df["rating"] == 0]
losers


,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,...,query,docID,reference_answer,type,queryID,query_norm,error_factual,error_czech,factual_comment,czech_comment


In [30]:
# Mean score per model (absolute quality)
# On average, how positively is this model judged?”
model_scores = (
    df
    .groupby("model")["rating"]
    .mean()
    .sort_values(ascending=False)
)
model_scores

model
llama3.3:latest    0.444444
gpt-oss-120b       0.388889
qwen3:32b          0.388889
phi4:14b-q8_0     -0.444444
Name: rating, dtype: float64

In [31]:
chat_model_scores = (
    df
    .groupby(["chat_id", "model"])["rating"]
    .sum()
    .reset_index()
)


In [32]:
df.columns

Index(['chat_id', 'chat_title', 'user_id', 'prompt', 'answer', 'model',
       'model_idx', 'siblings', 'rating', 'rating_comment', 'rating_reason',
       'is_rated_model', 'rating_zero_based', 'prompt_norm', 'query', 'docID',
       'reference_answer', 'type', 'queryID', 'query_norm', 'error_factual',
       'error_czech', 'factual_comment', 'czech_comment'],
      dtype='object')

In [35]:
prompt_model_ratings = (
    df
    .groupby(["prompt", "model"])["rating_zero_based"]
    .apply(list)
    .unstack(fill_value=[])
)

In [36]:
prompt_model_ratings

model,gpt-oss-120b,llama3.3:latest,phi4:14b-q8_0,qwen3:32b
prompt,,,,
Byl Word2Vec vyvinut výzkumníky z Microsoftu?,"[1, 0]","[1, 1]","[0, 0]","[1, 0]"
Jak lemmatizace zlepšuje latentní sémantickou analýzu?,"[0, 1]","[0, 1]","[0, 0]","[1, 1]"
"Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?","[1, 1]","[0, 0]","[0, 1]","[1, 1]"
Jaké doplňkové informace může lemmatizátor poskytovat?,"[0, 1]","[1, 1]","[1, 0]","[0, 1]"
Jaké jsou hlavní typy jazykových korpusů a jejich charakteristiky?,"[1, 1]","[1, 0]","[1, 1]","[0, 0]"
Jaké jsou tři historické fáze zpracování přirozeného jazyka uvedené v článku?,"[1, 1]","[1, 1]","[1, 0]","[1, 1]"
Jaké jsou úkoly morfologicko-lexikální analýzy v metodě automatického indexování zvané mozaika?,"[1, 1]","[1, 0]","[1, 0]","[1, 1]"
Jaký fonologický rys odlišuje foném /m/ od fonému /b/?,"[0, 0]","[1, 0]","[0, 0]","[0, 1]"
Jaký je hlavní rozdíl mezi úzkou a obecnou umělou inteligencí?,"[1, 1]","[1, 0]","[0, 0]","[1, 1]"


In [101]:
neg_reasons = df[df["rating"] == -1][
    ["prompt", "model", "rating_reason", "rating_comment"]
]

neg_reasons


,prompt,model,rating_reason,rating_comment
4,Jak lemmatizace zlepšuje latentní sémantickou ...,gpt-oss-120b,,c - špatný příklad různých tvarů stejného slov...
6,Jak lemmatizace zlepšuje latentní sémantickou ...,llama3.3:latest,,c - nečeské slova či části slov v textu
7,Jak lemmatizace zlepšuje latentní sémantickou ...,phi4:14b-q8_0,,"c - špatný příklad tvarů jednoho slova, špatně..."
9,Jaký výstup lemmatizátor vrátí pro tvar „barvě...,qwen3:32b,,f - podstatné jméno není mluvnická kategorie
10,Jaký výstup lemmatizátor vrátí pro tvar „barvě...,phi4:14b-q8_0,,f - podstatné jméno není mluvnická kategorie
13,Jaké doplňkové informace může lemmatizátor pos...,qwen3:32b,,f - špatné kaegorie u příkladu sloves
14,Jaké doplňkové informace může lemmatizátor pos...,gpt-oss-120b,,f- slovní typ místo slovní druh
15,Proč je v NLP nutná samostatná reprezentace pr...,gpt-oss-120b,,c - mathematický místo matemaický
16,Proč je v NLP nutná samostatná reprezentace pr...,llama3.3:latest,,c - čínský znak uprostřed
17,Proč je v NLP nutná samostatná reprezentace pr...,phi4:14b-q8_0,,f - 4 není elevantní


In [102]:

disagreements = (
    df
    .groupby(["prompt", "model"])
    .filter(lambda g: set(g["rating"]) == {1, -1})
)


In [103]:
for (prompt, model), g in disagreements.groupby(["prompt", "model"]):
    print("=" * 100)
    print(f"QUESTION:\n{prompt}")
    print(f"\nMODEL: {model}")
    print("ANNOTATOR RATINGS & REASONS:")

    for _, row in g.iterrows():
        print(f"  - rating: {row['rating']}")
        if row["rating_reason"]:
            print(f"    reason: {row['rating_reason']}")
        if row["rating_comment"]:
            print(f"    comment: {row['rating_comment']}")


QUESTION:
Byl Word2Vec vyvinut výzkumníky z Microsoftu?

MODEL: gpt-oss-120b
ANNOTATOR RATINGS & REASONS:
  - rating: 1
  - rating: -1
    comment: c - Kaím místo Kai
QUESTION:
Byl Word2Vec vyvinut výzkumníky z Microsoftu?

MODEL: qwen3:32b
ANNOTATOR RATINGS & REASONS:
  - rating: 1
  - rating: -1
    comment: c - Kaim Chen místo Kai Chenem, Ilyoum místo Ilya, Sutskever místo Sutskeverem, Deane místo Deanem
QUESTION:
Jak lemmatizace zlepšuje latentní sémantickou analýzu?

MODEL: gpt-oss-120b
ANNOTATOR RATINGS & REASONS:
  - rating: -1
    comment: c - špatný příklad různých tvarů stejného slova - barevný
  - rating: 1
QUESTION:
Jak lemmatizace zlepšuje latentní sémantickou analýzu?

MODEL: llama3.3:latest
ANNOTATOR RATINGS & REASONS:
  - rating: -1
    comment: c - nečeské slova či části slov v textu
  - rating: 1
QUESTION:
Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?

MODEL: phi4:14b-q8_0
ANNOTATOR RATINGS & REASONS:
  - 

In [104]:
def agreement_type(cell):
    if not cell:
        return "empty"
    s = set(cell)
    if s == {1}:
        return "agree_pos"
    if s == {-1}:
        return "agree_neg"
    return "disagree"

def agreement_style(cell):
    t = agreement_type(cell)

    if t == "agree_pos":
        return "background-color: lightgreen"
    if t == "agree_neg":
        return "background-color: lightcoral"
    if t == "disagree":
        return "background-color: lightblue; font-weight: bold"
    return ""


flat_cells = prompt_model_ratings.stack()

counts = flat_cells.apply(agreement_type).value_counts()

n_agree_pos = counts.get("agree_pos", 0)
n_agree_neg = counts.get("agree_neg", 0)
n_disagree  = counts.get("disagree", 0)

from IPython.display import display, HTML

title = f"""
<h3>
🟢 Agree +1: {n_agree_pos} &nbsp;&nbsp;
🔵 Disagree: {n_disagree} &nbsp;&nbsp;
🔴 Agree −1: {n_agree_neg}
</h3>
"""

display(HTML(title))

styled = prompt_model_ratings.style.applymap(agreement_style)
styled


/tmp/ipykernel_8727/2064362690.py:43: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = prompt_model_ratings.style.applymap(agreement_style)


model,gpt-oss-120b,llama3.3:latest,phi4:14b-q8_0,qwen3:32b
prompt,,,,
Byl Word2Vec vyvinut výzkumníky z Microsoftu?,"[1, -1]","[1, 1]","[-1, -1]","[1, -1]"
Jak lemmatizace zlepšuje latentní sémantickou analýzu?,"[-1, 1]","[-1, 1]","[-1, -1]","[1, 1]"
"Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?","[1, 1]","[-1, -1]","[-1, 1]","[1, 1]"
Jaké doplňkové informace může lemmatizátor poskytovat?,"[-1, 1]","[1, 1]","[1, -1]","[-1, 1]"
Jaké jsou hlavní typy jazykových korpusů a jejich charakteristiky?,"[1, 1]","[1, -1]","[1, 1]","[-1, -1]"
Jaké jsou tři historické fáze zpracování přirozeného jazyka uvedené v článku?,"[1, 1]","[1, 1]","[1, -1]","[1, 1]"
Jaké jsou úkoly morfologicko-lexikální analýzy v metodě automatického indexování zvané mozaika?,"[1, 1]","[1, -1]","[1, -1]","[1, 1]"
Jaký fonologický rys odlišuje foném /m/ od fonému /b/?,"[-1, -1]","[1, -1]","[-1, -1]","[-1, 1]"
Jaký je hlavní rozdíl mezi úzkou a obecnou umělou inteligencí?,"[1, 1]","[1, -1]","[-1, -1]","[1, 1]"


In [105]:
latex_table = (
    prompt_model_ratings
    .style
    .applymap(agreement_style)
    .to_latex(
        hrules=True,
        position="htbp",
        caption="Model agreement matrix",
        label="tab:agreement",
        convert_css=True
    )
)


/tmp/ipykernel_8727/3822130545.py:4: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(agreement_style)


In [106]:
with open("agreement_table.tex", "w") as f:
    f.write(latex_table)


In [107]:
df

,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,...,query,docID,reference_answer,type,queryID,query_norm,error_factual,error_czech,factual_comment,czech_comment
0,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...,0,0,,
1,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...,0,0,,
2,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza hraje důležitou...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
3,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza v metodě automa...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
4,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",-1,c - špatný příklad různých tvarů stejného slov...,...,Jak lemmatizace zlepšuje latentní sémantickou ...,17,snižuje počet slov na minimum převodem všech s...,direct,12,Jak lemmatizace zlepšuje latentní sémantickou ...,0,1,,c - špatný příklad různých tvarů stejného slov...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,a937c44c-54e7-48b2-a886-a0e8f4707b76,🔍 Unique NLP Representations Needed,b07241e5-a921-4445-bdfc-a876a7ceb08e,Proč je v NLP nutná samostatná reprezentace pr...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,None,...,Proč je v NLP nutná samostatná reprezentace pr...,4,Aby byla zajištěna přesnost a jednoznačnost re...,direct,5,Proč je v NLP nutná samostatná reprezentace pr...,0,0,,
140,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
141,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",1,"Něco navíc, rychlé",...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
142,e4b33b21-b23f-4fd4-a8f7-

In [108]:
def factual_agreement_type(cell):
    if not cell:
        return "empty"
    s = set(cell)
    if s == {0}:
        return "agree_no_factual"
    if s == {1}:
        return "agree_factual"
    return "disagree"

prompt_model_factual = (
    df
    .groupby(["prompt", "model"])["error_factual"]
    .apply(list)
    .unstack(fill_value=[])
)
def factual_agreement_style(cell):
    t = factual_agreement_type(cell)

    if t == "agree_no_factual":
        return "background-color: lightgreen"
    if t == "agree_factual":
        return "background-color: lightcoral"
    if t == "disagree":
        return "background-color: lightblue; font-weight: bold"
    return ""

flat_cells = prompt_model_factual.stack()

counts = flat_cells.apply(factual_agreement_type).value_counts()

n_agree_no_factual = counts.get("agree_no_factual", 0)
n_agree_factual    = counts.get("agree_factual", 0)
n_disagree         = counts.get("disagree", 0)

from IPython.display import display, HTML

title = f"""
<h3>
🟢 Agree: no factual error ({n_agree_no_factual}) &nbsp;&nbsp;
🔵 Disagree ({n_disagree}) &nbsp;&nbsp;
🔴 Agree: factual error ({n_agree_factual})
</h3>
"""

display(HTML(title))

styled = prompt_model_factual.style.applymap(factual_agreement_style)
styled


/tmp/ipykernel_8727/2796803316.py:48: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled = prompt_model_factual.style.applymap(factual_agreement_style)


model,gpt-oss-120b,llama3.3:latest,phi4:14b-q8_0,qwen3:32b
prompt,,,,
Byl Word2Vec vyvinut výzkumníky z Microsoftu?,"[0, 0]","[0, 0]","[1, 0]","[0, 0]"
Jak lemmatizace zlepšuje latentní sémantickou analýzu?,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"
"Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?","[0, 0]","[1, 1]","[0, 0]","[0, 0]"
Jaké doplňkové informace může lemmatizátor poskytovat?,"[1, 0]","[0, 0]","[0, 0]","[1, 0]"
Jaké jsou hlavní typy jazykových korpusů a jejich charakteristiky?,"[0, 0]","[0, 1]","[0, 0]","[1, 1]"
Jaké jsou tři historické fáze zpracování přirozeného jazyka uvedené v článku?,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"
Jaké jsou úkoly morfologicko-lexikální analýzy v metodě automatického indexování zvané mozaika?,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"
Jaký fonologický rys odlišuje foném /m/ od fonému /b/?,"[0, 0]","[0, 0]","[1, 0]","[0, 0]"
Jaký je hlavní rozdíl mezi úzkou a obecnou umělou inteligencí?,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"


In [109]:
def factual_accuracy_cell(cell):
    if not cell:
        return None
    # 0 = no error → 1 point
    # 1 = error → 0 points
    return sum(1 - x for x in cell) / len(cell)


In [110]:
factual_accuracy_matrix = prompt_model_factual.applymap(factual_accuracy_cell)
factual_accuracy_matrix


/tmp/ipykernel_8727/558975624.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  factual_accuracy_matrix = prompt_model_factual.applymap(factual_accuracy_cell)


model,gpt-oss-120b,llama3.3:latest,phi4:14b-q8_0,qwen3:32b
prompt,,,,
Byl Word2Vec vyvinut výzkumníky z Microsoftu?,1.0,1.0,0.5,1.0
Jak lemmatizace zlepšuje latentní sémantickou analýzu?,1.0,1.0,1.0,1.0
"Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?",1.0,0.0,1.0,1.0
Jaké doplňkové informace může lemmatizátor poskytovat?,0.5,1.0,1.0,0.5
Jaké jsou hlavní typy jazykových korpusů a jejich charakteristiky?,1.0,0.5,1.0,0.0
Jaké jsou tři historické fáze zpracování přirozeného jazyka uvedené v článku?,1.0,1.0,1.0,1.0
Jaké jsou úkoly morfologicko-lexikální analýzy v metodě automatického indexování zvané mozaika?,1.0,1.0,1.0,1.0
Jaký fonologický rys odlišuje foném /m/ od fonému /b/?,1.0,1.0,0.5,1.0
Jaký je hlavní rozdíl mezi úzkou a obecnou umělou inteligencí?,1.0,1.0,1.0,1.0


In [111]:
factual_accuracy_per_model = factual_accuracy_matrix.mean().sort_values(ascending=False)
factual_accuracy_per_model


model
gpt-oss-120b       0.916667
llama3.3:latest    0.888889
qwen3:32b          0.833333
phi4:14b-q8_0      0.777778
dtype: float64

In [112]:
df

,chat_id,chat_title,user_id,prompt,answer,model,model_idx,siblings,rating,rating_comment,...,query,docID,reference_answer,type,queryID,query_norm,error_factual,error_czech,factual_comment,czech_comment
0,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...,0,0,,
1,e4b33b21-b23f-4fd4-a8f7-597b75893dc8,🔍 Historické Fáze NLP,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou tři historické fáze zpracování přiro...,Tři historické fáze zpracování přirozeného jaz...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou tři historické fáze zpracování přiro...,4,"symbolické NLP (50. až 90. léta 20. stol.), st...",direct,3,Jaké jsou tři historické fáze zpracování přiro...,0,0,,
2,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza hraje důležitou...,llama3.3:latest,1,"[phi4:14b-q8_0, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
3,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,Morfologicko-lexikální analýza v metodě automa...,phi4:14b-q8_0,0,"[llama3.3:latest, qwen3:32b, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
4,fe8a0d00-9d6a-40bd-9e39-d0bf4080cb5a,Lemmatization in LSA 📊,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jak lemmatizace zlepšuje latentní sémantickou ...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",-1,c - špatný příklad různých tvarů stejného slov...,...,Jak lemmatizace zlepšuje latentní sémantickou ...,17,snižuje počet slov na minimum převodem všech s...,direct,12,Jak lemmatizace zlepšuje latentní sémantickou ...,0,1,,c - špatný příklad různých tvarů stejného slov...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,a937c44c-54e7-48b2-a886-a0e8f4707b76,🔍 Unique NLP Representations Needed,b07241e5-a921-4445-bdfc-a876a7ceb08e,Proč je v NLP nutná samostatná reprezentace pr...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,None,...,Proč je v NLP nutná samostatná reprezentace pr...,4,Aby byla zajištěna přesnost a jednoznačnost re...,direct,5,Proč je v NLP nutná samostatná reprezentace pr...,0,0,,
140,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",qwen3:32b,2,"[phi4:14b-q8_0, llama3.3:latest, gpt-oss-120b]",1,,...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
141,231628ce-773e-4f35-bd94-ecbfcd6abdf8,🔍 Morfologicko-Lexikální Analýza,b07241e5-a921-4445-bdfc-a876a7ceb08e,Jaké jsou úkoly morfologicko-lexikální analýzy...,"<details type=""reasoning"" done=""true"" duration...",gpt-oss-120b,3,"[phi4:14b-q8_0, llama3.3:latest, qwen3:32b]",1,"Něco navíc, rychlé",...,Jaké jsou úkoly morfologicko‑lexikální analýzy...,4,Vyloučení nevýznamových termínů pomocí negativ...,direct,4,Jaké jsou úkoly morfologicko-lexikální analýzy...,0,0,,
142,e4b33b21-b23f-4fd4-a8f7-

In [113]:
consensus = (
    df
    .groupby(["prompt", "model"])
    .agg(
        ratings=("rating", list),
        rating_mean=("rating", "mean"),
        error_factual=("error_factual", list),
        error_czech=("error_czech", list),
        n_annotators=("rating", "size"),
    )
)
consensus

ratings  \
prompt                                             model                       
Byl Word2Vec vyvinut výzkumníky z Microsoftu?      gpt-oss-120b      [1, -1]   
                                                   llama3.3:latest    [1, 1]   
                                                   phi4:14b-q8_0    [-1, -1]   
                                                   qwen3:32b         [1, -1]   
Jak lemmatizace zlepšuje latentní sémantickou a... gpt-oss-120b      [-1, 1]   
...                                                                      ...   
Pro kterou společnost pracuje Greg Corrado podl... qwen3:32b         [1, -1]   
Proč je v NLP nutná samostatná reprezentace pro... gpt-oss-120b      [-1, 1]   
                                                   llama3.3:latest  [-1, -1]   
                                                   phi4:14b-q8_0    [-1, -1]   
                                                   qwen3:32b          [1, 1]   

                                                                    rating_mean  \
prompt                                             model                          
Byl Word2Vec vyvinut výzkumníky z Microsoftu?      gpt-oss-120b             0.0   
                                                   llama3.3:latest          1.0   
                                                   phi4:14b-q8_0           -1.0   
                                                   qwen3:32b                0.0   
Jak lemmatizace zlepšuje latentní sémantickou a... gpt-oss-120b             0.0   
...                                                                         ...   
Pro kterou společnost pracuje Greg Corrado podl... qwen3:32b                0.0   
Proč je v NLP nutná samostatná reprezentace pro... gpt-oss-120b             0.0   
                                                   llama3.3:latest         -1.0   
                                                   phi4:14b-q8_0           -1.0   
                                                   qwen3:32b                1.0   

                                                                   error_factual  \
prompt                                             model                           
Byl Word2Vec vyvinut výzkumníky z Microsoftu?      gpt-oss-120b           [0, 0]   
                                                   llama3.3:latest        [0, 0]   
                                                   phi4:14b-q8_0          [1, 0]   
                                                   qwen3:32b              [0, 0]   
Jak lemmatizace zlepšuje latentní sémantickou a... gpt-oss-120b           [0, 0]   
...                                                                          ...   
Pro kterou společnost pracuje Greg Corrado podl... qwen3:32b              [0, 0]   
Proč je v NLP nutná samostatná reprezentace pro... gpt-oss-120b           [0, 0]   
                                                   llama3.3:latest        [0, 1]   
                                                   phi4:14b-q8_0          [1, 1]   
                                                   qwen3:32b              [0, 0]   

                                                                   error_czech  \
prompt                                             model                         
Byl Word2Vec vyvinut výzkumníky z Microsoftu?      gpt-oss-120b         [0, 1]   
                                                   llama3.3:latest      [0, 0]   
                                                   phi4:14b-q8_0        [1, 1]   
                                                   qwen3:32b            [0, 1]   
Jak lemmatizace zlepšuje latentní sémantickou a... gpt-oss-120b         [1, 0]   
...                                                                        ...   
Pro kterou společnost pracuje Greg Corrado podl... qwen3:32b            [0, 1]   
Proč je v NLP nutná samostatná reprezentace pro... gpt-oss-120b         [1, 0]   
                                   

In [114]:
df.columns

Index(['chat_id', 'chat_title', 'user_id', 'prompt', 'answer', 'model',
       'model_idx', 'siblings', 'rating', 'rating_comment', 'rating_reason',
       'is_rated_model', 'rating_zero_based', 'prompt_norm', 'query', 'docID',
       'reference_answer', 'type', 'queryID', 'query_norm', 'error_factual',
       'error_czech', 'factual_comment', 'czech_comment'],
      dtype='object')

In [115]:
df[[ 'queryID','model','answer']].sort_values(by='queryID')

,queryID,model,answer
99,0,gpt-oss-120b,"<details type=""reasoning"" done=""true"" duration..."
100,0,qwen3:32b,"<details type=""reasoning"" done=""true"" duration..."
102,0,llama3.3:latest,"Neuronová síť, která vznikla jako simulace mod..."
66,0,phi4:14b-q8_0,"Neuronovou síť, která vznikla jako simulace fy..."
137,0,phi4:14b-q8_0,"Neuronová síť, která vznikla jako simulace fyz..."
...,...,...,...
45,17,gpt-oss-120b,"<details type=""reasoning"" done=""true"" duration..."
32,17,phi4:14b-q8_0,Hlavní typy jazykových korpusů lze rozdělit na...
33,17,qwen3:32b,"<details type=""reasoning"" done=""true"" duration..."
42,17,phi4:14b-q8_0,Hlavními typy jazykových korpusů jsou následuj...


In [116]:
df[[ 'queryID','model','answer']].sort_values(by='queryID').iloc[3].answer

'Neuronovou síť, která vznikla jako simulace fyziologického modelu rozpoznávání vzorů na sítnici lidského oka, nazývají Grossbergova neuronová síť. Tento typ sítě byl navržen stevenem grossbergem pro analýzu procesů zpracování informací v nervovém systému [1] **(anglicky).**'

In [117]:
df[[ 'queryID','model','answer']].sort_values(by='queryID').iloc[4].answer


'Neuronová síť, která vznikla jako simulace fyziologického modelu rozpoznávání vzorů na sítnici lidského oka, se jmenuje Grossbergova síť. Toto uvedení je podloženo kontextem [1], ve kterém se uvádí: "například Grossbergova síť[6] vznikla původně jako simulace fyziologického modelu rozpoznávání vzorů na sítnici lidského oka."'

In [36]:
df[['queryID', 'prompt_norm']]

,queryID,prompt_norm
0,3,Jaké jsou tři historické fáze zpracování přiro...
1,3,Jaké jsou tři historické fáze zpracování přiro...
2,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
3,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
4,12,Jak lemmatizace zlepšuje latentní sémantickou ...
...,...,...
139,5,Proč je v NLP nutná samostatná reprezentace pr...
140,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
141,4,Jaké jsou úkoly morfologicko-lexikální analýzy...
142,3,Jaké jsou tři historické fáze zpracování přiro...


In [38]:
grouped = (
    df[df["queryID"].isin([0])]
    .groupby(["model", "prompt_norm"])["answer"]
    .apply(list)
    .reset_index()
)

for _, row in grouped.iterrows():
    print("=" * 80)
    print(f"MODEL: {row['model']}")
    print(f"PROMPT: {row['prompt_norm']}\n")
    for i, ans in enumerate(row["answer"], 1):
        print(f"[Answer {i}]\n{ans}\n")


MODEL: gpt-oss-120b
PROMPT: Jak se jmenuje neuronová síť, která vznikla jako simulace modelu rozpoznávání vzorů na sítnici lidského oka?

[Answer 1]
<details type="reasoning" done="true" duration="1">
<summary>Thought for 1 seconds</summary>
> We need to answer in Czech. Use context. The context contains a mention: "Například Grossbergova síť vznikla původně jako simulace fyziologického modelu rozpoznávání vzorů na sítnici lidského oka." So the answer: Grossbergova síť. Provide citation with id. The source with id "1" contains that text. Also source id "2" also mentions Grossbergova neuronová síť. But we need to use inline citations only when source has id attribute. Those sources have id "1" and "2". Both have id attributes. So we can cite one. Use maybe [1] referencing the first source. Provide answer: "Jedná se o Grossbergovu síť." Add citation.
</details>
Jedná se o **Grossbergovu síť** – neuronovou síť, která byla původně vytvořena jako simulace fyziologického modelu rozpoznávání 

agreement



In [37]:
ratings_pairwise = (
    df
    .sort_values(["prompt", "model"])
    .groupby(["prompt", "model"])["rating_zero_based"]
    .apply(list)
)

# sanity check
assert ratings_pairwise.apply(len).eq(2).all()


In [38]:
ratings_pairwise

prompt                                                                                     model          
Byl Word2Vec vyvinut výzkumníky z Microsoftu?                                              gpt-oss-120b       [1, 0]
                                                                                           llama3.3:latest    [1, 1]
                                                                                           phi4:14b-q8_0      [0, 0]
                                                                                           qwen3:32b          [1, 0]
Jak lemmatizace zlepšuje latentní sémantickou analýzu?                                     gpt-oss-120b       [0, 1]
                                                                                                               ...  
Pro kterou společnost pracuje Greg Corrado podle textu?                                    qwen3:32b          [1, 0]
Proč je v NLP nutná samostatná reprezentace pro každý významový prvek přir

Cohen’s κ within each model

In [40]:
from sklearn.metrics import cohen_kappa_score

within_model_kappa = (
    ratings_pairwise
    .groupby("model")
    .apply(lambda s: cohen_kappa_score(
        [x[0] for x in s],
        [x[1] for x in s]
    ))
    .rename("cohen_kappa")
    .reset_index()
)

within_model_kappa


,model,cohen_kappa
0,gpt-oss-120b,0.347826
1,llama3.3:latest,0.181818
2,phi4:14b-q8_0,0.181818
3,qwen3:32b,0.112676


In [46]:
df.groupby("model")["rating_zero_based"].value_counts(normalize=True)


model            rating_zero_based
gpt-oss-120b     1                    0.694444
                 0                    0.305556
llama3.3:latest  1                    0.722222
                 0                    0.277778
phi4:14b-q8_0    0                    0.722222
                 1                    0.277778
qwen3:32b        1                    0.694444
                 0                    0.305556
Name: proportion, dtype: float64

In [47]:
raw_agreement = (
    ratings_pairwise
    .groupby("model")
    .apply(lambda s: sum(x[0] == x[1] for x in s) / len(s))
    .rename("percent_agreement")
    .reset_index()
)

raw_agreement


,model,percent_agreement
0,gpt-oss-120b,0.722222
1,llama3.3:latest,0.666667
2,phi4:14b-q8_0,0.666667
3,qwen3:32b,0.611111


In [41]:
df_majority = (
    ratings_pairwise
    .apply(lambda x: int(sum(x) >= 1))
    .rename("rating")
    .reset_index()
)


In [42]:
ratings_wide = (
    df_majority
    .pivot(index="prompt", columns="model", values="rating")
    .dropna()
)


In [43]:
import itertools
import pandas as pd

kappa_records = []

for m1, m2 in itertools.combinations(ratings_wide.columns, 2):
    kappa = cohen_kappa_score(
        ratings_wide[m1],
        ratings_wide[m2]
    )
    kappa_records.append({
        "model_1": m1,
        "model_2": m2,
        "cohen_kappa": kappa
    })

between_model_kappa = pd.DataFrame(kappa_records)
between_model_kappa


,model_1,model_2,cohen_kappa
0,gpt-oss-120b,llama3.3:latest,-0.153846
1,gpt-oss-120b,phi4:14b-q8_0,0.275862
2,gpt-oss-120b,qwen3:32b,-0.153846
3,llama3.3:latest,phi4:14b-q8_0,-0.022727
4,llama3.3:latest,qwen3:32b,-0.125000
5,phi4:14b-q8_0,qwen3:32b,-0.022727


In [44]:
per_model_agreement = (
    between_model_kappa
    .melt(id_vars="cohen_kappa", value_vars=["model_1", "model_2"], value_name="model")
    .groupby("model")["cohen_kappa"]
    .mean()
    .sort_values(ascending=False)
)

per_model_agreement


model
phi4:14b-q8_0      0.076803
gpt-oss-120b      -0.010610
llama3.3:latest   -0.100524
qwen3:32b         -0.100524
Name: cohen_kappa, dtype: float64

In [45]:
per_model_agreement = (
    between_model_kappa
    .melt(id_vars="cohen_kappa", value_vars=["model_1", "model_2"], value_name="model")
    .groupby("model")["cohen_kappa"]
    .mean()
    .sort_values(ascending=False)
)

per_model_agreement


model
phi4:14b-q8_0      0.076803
gpt-oss-120b      -0.010610
llama3.3:latest   -0.100524
qwen3:32b         -0.100524
Name: cohen_kappa, dtype: float64